In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("../data/raw/BTCUSDT.csv")
df

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,2017-08-17 04:00:59.999,7.564907e+03,3,0.075183,320.390851,0
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:01:59.999,0.000000e+00,0,0.000000,0.000000,0
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,2017-08-17 04:02:59.999,1.117543e+03,2,0.261074,1117.542921,0
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,2017-08-17 04:03:59.999,5.117185e+01,3,0.012008,51.171852,0
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,2017-08-17 04:04:59.999,5.999993e+02,1,0.140796,599.999338,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3575465,2024-06-10 02:56:00,69721.42,69721.43,69700.00,69717.69,21.915850,2024-06-10 02:56:59.999000,1.527848e+06,563,7.493360,522364.476497,0
3575466,2024-06-10 02:57:00,69717.69,69717.70,69715.89,69715.89,5.537070,2024-06-10 02:57:59.999000,3.860293e+05,221,1.261270,87932.813545,0
3575467,2024-06-10 02:58:00,69715.89,69724.93,69715.89,69724.92,9.078770,2024-06-10 02:58:59.999000,6.329483e+05,340,5.296120,369236.976039,0
3575468,2024-06-10 02:59:00,69724.92,69724.93,69715.32,69716.00,12.614700,2024-06-10 02:59:59.999000,8.794528e+05,411,8.735730,609015.026616,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3575470 entries, 0 to 3575469
Data columns (total 12 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   timestamp                     object 
 1   open                          float64
 2   high                          float64
 3   low                           float64
 4   close                         float64
 5   volume                        float64
 6   close_time                    object 
 7   quote_asset_volume            float64
 8   number_of_trades              int64  
 9   taker_buy_base_asset_volume   float64
 10  taker_buy_quote_asset_volume  float64
 11  ignore                        int64  
dtypes: float64(8), int64(2), object(2)
memory usage: 327.3+ MB
